In [2]:
import psycopg2
import os
import pandas as pd
import pickle
import numpy as np

from lxml import etree
from psycopg2 import sql
from pymongo import MongoClient

## создание датасета

- на этапе соединения - 1 337 681 сырых строк из MongoDB

In [389]:
with open('parsed_data/mongo_data.pickle', 'rb') as f:
    mongo_data = pickle.load(f)

In [390]:
mongo_data

,_id,id_org,shortName,ogrn,index,region,district,city,settlement,okved,...,2020_2350,2021_2300,2020_2300,2021_2410,2020_2410,2021_2400,2020_2400,2021_2500,2020_2500,2021_1220
0,3447001553,5581428,"ООО ""МОДЕРН-БОН""",1023404293505,400066,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,0.0,NaN,NaN,138.0,0.0,1270.0,0.0,NaN,NaN,NaN
1,3447003470,5573882,"ООО ""МАРКО""",1023404287829,400079,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,NaN,-29.0,30.0,NaN,NaN,-29.0,30.0,-29.0,30.0,NaN
2,3447003247,6838461,"ООО ""ОКСИ""",1023444287855,400057,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,83.0,NaN,NaN,152.0,113.0,1099.0,691.0,NaN,NaN,NaN
3,3447003416,6838444,"ОАО ""ФИРМА ЖБИ-6""",1023444287580,400057,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,1586.0,24837.0,11070.0,-5181.0,-2283.0,19656.0,8787.0,17783.0,7599.0,NaN
4,3447002444,5581537,"ПК ""МОНОЛИТ""",1023404293725,400059,ВОЛГОГРАДСКАЯ,None,ВОЛГОГРАД,None,None,...,NaN,0.0,4.0,NaN,NaN,0.0,4.0,0.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337676,8901027500,7876989,"ООО ""ОТКРЫТЫЕ ТЕХНОЛОГИИ""",1138901000260,629008,ЯМАЛО-НЕНЕЦКИЙ,None,Салехард,None,None,...,NaN,NaN,NaN,NaN,NaN,-1297.0,-1297.0,-1297.0,-1297.0,NaN
1337677,8901034465,9973663,"ООО ""СТК ОРБИТА""",1168901056598,629404,ЯМАЛО-НЕНЕЦКИЙ,None,ЛАБЫТНАНГИ,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1337678,8901036769,10699545,"ООО ""СЕВУРАЛМЕТ""",1188901002344,620014,СВЕРДЛОВСКАЯ,None,ЕКАТЕРИНБУРГ,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1337679,8901035081,10153373,"ООО ""ЯМАЛ ФИШ""",1178901001388,629008,ЯМАЛО-НЕНЕЦКИЙ,None,САЛЕХАРД,None,None,...,-575.0,NaN,NaN,0.0,0.0,383.0,-1200.0,NaN,NaN,NaN


- оставляю нужные столбцы, привожу типы - чтобы уменьшить размер
- shortname и _id в датасет не попадут, они просто для удобства

In [391]:
columns = ['_id', 'shortName', 'index', 'region', 'city', 'settlement', 'okved2', 'okopf',
           'statusCode', 'registrationDate', 'authorizedCapital', '2021_1150', '2020_1150',
           '2019_1150', '2021_1170', '2020_1170', '2019_1170', '2021_1210', '2020_1210',
           '2019_1210', '2021_1230', '2020_1230', '2019_1230', '2021_1250', '2020_1250',
           '2019_1250', '2021_1600', '2020_1600', '2019_1600', '2021_1350', '2020_1350',
           '2019_1350', '2021_1360', '2020_1360', '2019_1360', '2021_1300', '2020_1300',
           '2019_1300', '2021_1520', '2020_1520', '2019_1520', '2021_1700', '2020_1700',
           '2019_1700', '2021_2110', '2020_2110', '2021_2120', '2020_2120', '2021_2330',
           '2020_2330', '2021_2340', '2020_2340', '2021_2350', '2020_2350', '2021_2410',
           '2020_2410', '2021_2400', '2020_2400',
          ]
mongo_data = mongo_data[columns]

In [392]:
mongo_data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337681 entries, 0 to 1337680
Data columns (total 58 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   _id                1337681 non-null  object 
 1   shortName          1337681 non-null  object 
 2   index              1333535 non-null  object 
 3   region             1337642 non-null  object 
 4   city               889512 non-null   object 
 5   settlement         139302 non-null   object 
 6   okved2             1336696 non-null  object 
 7   okopf              1317004 non-null  float64
 8   statusCode         1337681 non-null  object 
 9   registrationDate   1337680 non-null  object 
 10  authorizedCapital  1281864 non-null  float64
 11  2021_1150          579961 non-null   float64
 12  2020_1150          477848 non-null   float64
 13  2019_1150          443712 non-null   float64
 14  2021_1170          155110 non-null   float64
 15  2020_1170          108221 non-nu

- на всякий случай чистим инн

In [393]:
mongo_data._id = mongo_data._id.apply(lambda x: x.strip())
mongo_data._id.nunique(), mongo_data._id.size

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


(1337681, 1337681)

- окопф - заполняем пропуски 0 и приводим к object
- на самом деле оказалось, что удобнее (быстрее) работать как с цифрой, а потеря первого нуля не критична

In [394]:
mongo_data.okopf = mongo_data.okopf.fillna(0).astype('int32').astype('object')

- дату - к datetime

In [395]:
mongo_data.registrationDate = pd.to_datetime(mongo_data.registrationDate)

- посмотрим, удастся ли привести какой-нибудь float64 к float32

In [396]:
for col in mongo_data.columns[10:]:
    mongo_data[col] = pd.to_numeric(mongo_data[col], downcast='float')

C:\Users\013\AppData\Local\Temp/ipykernel_3852/315581999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mongo_data[col] = pd.to_numeric(mongo_data[col], downcast='float')


In [397]:
mongo_data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337681 entries, 0 to 1337680
Data columns (total 58 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   _id                1337681 non-null  object        
 1   shortName          1337681 non-null  object        
 2   index              1333535 non-null  object        
 3   region             1337642 non-null  object        
 4   city               889512 non-null   object        
 5   settlement         139302 non-null   object        
 6   okved2             1336696 non-null  object        
 7   okopf              1337681 non-null  object        
 8   statusCode         1337681 non-null  object        
 9   registrationDate   1337680 non-null  datetime64[ns]
 10  authorizedCapital  1281864 non-null  float32       
 11  2021_1150          579961 non-null   float32       
 12  2020_1150          477848 non-null   float32       
 13  2019_1150          443712 n

#### - дальше забираем вторую большую таблицу - реестр мсп из PostgreSQL. 

- нужна не вся таблица, а только те инн, которые есть в Mongo - но, кажется, отбросить лишние строки будет быстрее в python.

In [3]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='', host="localhost", port=5432)

In [7]:
connect.close()

In [4]:
columns = [
    'inn', 
    'n_employees', 
    'is_soc_enterprise', 
    'newly_establ', 
    'subj_cat', 
    'subj_form', 
    'inclusion_date',  # отличается от registration_date! - сравнить!
    'region_code', 
    'n_okovd_add',  
#     'okovd_add_vers',  # только убедиться, что везде одно значение! - везде, где есть, одно
#     'okovd_add_code',  # можно придумать много интересных признаков
    'n_licenses',
    'n_products',
#     'products_code'  # то же самое
    'is_high_tech',
    'n_contracts',
    'n_agreements'
]

In [5]:
%%time
reestr_msp = pd.read_sql(sql.SQL('SELECT {} FROM reestr_msp;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

Wall time: 16 s


In [402]:
reestr_msp.head()

,inn,n_employees,is_soc_enterprise,newly_establ,subj_cat,subj_form,inclusion_date,region_code,n_okovd_add,n_licenses,n_products,is_high_tech,n_contracts,n_agreements
0,3447001553,1.0,0,0,1,1,2016-08-01,34,29,0,0,0,0,0
1,3447003470,2.0,0,0,1,1,2016-08-01,34,6,0,0,0,0,0
2,3447003247,18.0,0,0,1,1,2016-08-01,34,24,1,0,0,0,0
3,3447003416,165.0,0,0,3,1,2018-08-10,34,9,2,0,0,0,0
4,3447002444,NaN,0,0,1,1,2016-08-01,34,0,0,0,0,0,0


- здесь инн чистые - это уникальный ключ
- оставляем только нужные строки

In [403]:
reestr_msp.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332299 entries, 0 to 2332298
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   inn                2332299 non-null  object 
 1   n_employees        1938849 non-null  float64
 2   is_soc_enterprise  2332299 non-null  int64  
 3   newly_establ       2332299 non-null  int64  
 4   subj_cat           2332299 non-null  int64  
 5   subj_form          2332299 non-null  int64  
 6   inclusion_date     2332299 non-null  object 
 7   region_code        2332299 non-null  int64  
 8   n_okovd_add        2332299 non-null  int64  
 9   n_licenses         2332299 non-null  int64  
 10  n_products         2332299 non-null  int64  
 11  is_high_tech       2332299 non-null  int64  
 12  n_contracts        2332299 non-null  int64  
 13  n_agreements       2332299 non-null  int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 249.1+ MB


In [404]:
# они уникальны
actual_inn = tuple(mongo_data._id)

- а это очень интересно, как так вышло, что в реестре меньше инн, чем в данных из интернета: должно быть строго наоборот (так как паук идёт по списку из реестра), если инн нет в реестре, то как его можно было найти на сайте bfo?
- но при обработке xml некоторые данные отбрасывались как некорректные

In [405]:
len(actual_inn), reestr_msp.loc[reestr_msp.inn.isin(actual_inn)].shape

(1337681, (1337248, 14))

In [406]:
reestr_msp = reestr_msp.loc[reestr_msp.inn.isin(actual_inn)]

reestr_msp.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337248 entries, 0 to 2330013
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   inn                1337248 non-null  object 
 1   n_employees        1284409 non-null  float64
 2   is_soc_enterprise  1337248 non-null  int64  
 3   newly_establ       1337248 non-null  int64  
 4   subj_cat           1337248 non-null  int64  
 5   subj_form          1337248 non-null  int64  
 6   inclusion_date     1337248 non-null  object 
 7   region_code        1337248 non-null  int64  
 8   n_okovd_add        1337248 non-null  int64  
 9   n_licenses         1337248 non-null  int64  
 10  n_products         1337248 non-null  int64  
 11  is_high_tech       1337248 non-null  int64  
 12  n_contracts        1337248 non-null  int64  
 13  n_agreements       1337248 non-null  int64  
dtypes: float64(1), int64(11), object(2)
memory usage: 153.0+ MB


- в n_employees заполняю Nan как -1 и заодно изменю тип

In [407]:
reestr_msp.n_employees.min(), reestr_msp.n_employees.max()

(0.0, 8617.0)

In [408]:
reestr_msp.n_employees = pd.to_numeric(reestr_msp.n_employees.fillna(-1), downcast='integer')

In [409]:
n_columns = ['is_soc_enterprise', 'newly_establ', 'subj_cat', 'subj_form', 'region_code', 
             'n_products', 'is_high_tech', 'n_okovd_add', 'n_licenses', 'n_contracts', 
             'n_agreements']

In [410]:
for col in n_columns:
    reestr_msp[col] = pd.to_numeric(reestr_msp[col], downcast='integer')

In [6]:
reestr_msp.inclusion_date = pd.to_datetime(reestr_msp.inclusion_date)

In [411]:
reestr_msp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337248 entries, 0 to 2330013
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   inn                1337248 non-null  object
 1   n_employees        1337248 non-null  int16 
 2   is_soc_enterprise  1337248 non-null  int8  
 3   newly_establ       1337248 non-null  int8  
 4   subj_cat           1337248 non-null  int8  
 5   subj_form          1337248 non-null  int8  
 6   inclusion_date     1337248 non-null  object
 7   region_code        1337248 non-null  int8  
 8   n_okovd_add        1337248 non-null  int16 
 9   n_licenses         1337248 non-null  int16 
 10  n_products         1337248 non-null  int8  
 11  is_high_tech       1337248 non-null  int8  
 12  n_contracts        1337248 non-null  int16 
 13  n_agreements       1337248 non-null  int8  
dtypes: int16(4), int8(8), object(2)
memory usage: 51.0+ MB


#### - теперь объединим две основные таблицы 


- не знаю, оставлять ли строки, для которых нет соответствия в reestr_msp? 
- пока оставлю и посмотрю на статус этих организаций - так как у меня и так дисбаланс

In [412]:
mongo_data.rename(columns={'_id': 'inn'}, inplace=True)

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


- в итоге - не беру пропуски (inner а не left)

In [413]:
%%time
data = mongo_data.merge(reestr_msp, how='inner', on='inn')

Wall time: 2.97 s


- здесь много INACTIVE
- но 400+ пропусков, для которых нет данных из реестра из паука

In [55]:
# data.loc[data.n_employees.isna(), 'statusCode'].value_counts()

ACTIVE               372
INACTIVE              58
LIQUIDATION_STAGE      3
Name: statusCode, dtype: int64

- но соотношение-то не сильно меняется...

In [57]:
# с пропусками в реестре
# mongo_data.statusCode.value_counts(normalize=True)

ACTIVE                  0.978073
LIQUIDATION_STAGE       0.016793
INACTIVE                0.003017
REORGANIZATION_STAGE    0.002117
Name: statusCode, dtype: float64

In [56]:
# без пропусков в реестре
# data.loc[~data.n_employees.isna()].statusCode.value_counts(normalize=True)

ACTIVE                  0.978111
LIQUIDATION_STAGE       0.016796
INACTIVE                0.002975
REORGANIZATION_STAGE    0.002118
Name: statusCode, dtype: float64

#### - идём дальше, support_msp 
- здесь надо аггрегировать

In [415]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)

In [418]:
connect.close()

In [416]:
columns = [
    'inn', 
    'support_date', 
    'is_breach',  # их и так немного - 890, а при разбивке на периоды - и того меньше...
    'form_code', 
    'support_volume', 
    'support_unit', 
]

In [417]:
%%time
support_msp = pd.read_sql(sql.SQL('SELECT {} FROM support_msp;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

Wall time: 4.27 s


In [419]:
support_msp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109362 entries, 0 to 1109361
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   inn             1109362 non-null  object 
 1   support_date    1109362 non-null  object 
 2   is_breach       1109362 non-null  int64  
 3   form_code       1109362 non-null  object 
 4   support_volume  1109362 non-null  float64
 5   support_unit    1109362 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 50.8+ MB


- сразу удаляем лишние строки

In [421]:
support_msp = support_msp.loc[support_msp.inn.isin(actual_inn)]

support_msp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 753236 entries, 3 to 1109305
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   inn             753236 non-null  object 
 1   support_date    753236 non-null  object 
 2   is_breach       753236 non-null  int64  
 3   form_code       753236 non-null  object 
 4   support_volume  753236 non-null  float64
 5   support_unit    753236 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 40.2+ MB


- добавляем год - чтобы взять только нужный период

In [422]:
support_msp['support_year'] = pd.to_datetime(support_msp.support_date).dt.year

- оставляем только нужные периоды (по которым есть данные!)

In [423]:
support_msp.support_year.unique()

array([2020, 2021, 2022, 2019, 2023], dtype=int64)

In [424]:
support_msp = support_msp.loc[support_msp.support_year.isin((2021, 2020, 2019))]

- аггрегируем: берём только support_unit = 1

In [425]:
agg_support = pd.pivot_table(support_msp.loc[support_msp.support_unit == 1], values='support_volume', 
               index='inn', columns=['form_code', 'support_year'], aggfunc=sum).reset_index()

In [426]:
agg_support.head(3)

form_code            inn 0101           0102           0103               \
support_year             2019 2020 2021 2019 2020 2021 2019         2020   
0             0101000021  NaN  NaN  NaN  NaN  NaN  NaN  NaN    145560.00   
1             0101000832  NaN  NaN  NaN  NaN  NaN  NaN  NaN    509460.00   
2             0101003086  NaN  NaN  NaN  NaN  NaN  NaN  NaN  15177215.01   

form_code              0105            
support_year      2021 2019 2020 2021  
0                  NaN  NaN  NaN  NaN  
1             243048.0  NaN  NaN  NaN  
2                  NaN  NaN  NaN  NaN

- избавимся от мультииндекса в столбцах

In [427]:
agg_support.columns = ['inn'] + [f'{el[0]}_{str(el[1])}' for el in agg_support.columns.values[1:]]

In [428]:
agg_support.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155313 entries, 0 to 155312
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        155313 non-null  object 
 1   0101_2019  20 non-null      float64
 2   0101_2020  27 non-null      float64
 3   0101_2021  8 non-null       float64
 4   0102_2019  3501 non-null    float64
 5   0102_2020  4080 non-null    float64
 6   0102_2021  6547 non-null    float64
 7   0103_2019  10883 non-null   float64
 8   0103_2020  121553 non-null  float64
 9   0103_2021  82513 non-null   float64
 10  0105_2019  109 non-null     float64
 11  0105_2020  169 non-null     float64
 12  0105_2021  132 non-null     float64
dtypes: float64(12), object(1)
memory usage: 15.4+ MB


- некоторую поддержку сразу нет смысла брать

        0101 - Инвестиции в капитал, убираем, было бы полезно для узкой выборки!
        0102 - гарантии и поручительство
        0103 - субсидии и гранты
        0105 - финансовая аренда - сомнительно, но пока оставим

- на самом деле уже здесь есть вопросы для анализа:

        - кому и сколько инвестируют в капитал?
        - код 0104 - с процентами - финансовая аренда: кому и сколько?

In [429]:
agg_col = ['0102_2019', '0102_2020', '0102_2021', '0103_2019', '0103_2020', 
           '0103_2021', '0105_2019', '0105_2020', '0105_2021']
agg_support = agg_support[['inn'] + agg_col]

- смотрим, можно ли привести типы:

In [430]:
for col in agg_support.columns[1:]:
    agg_support[col] = pd.to_numeric(agg_support[col], downcast='float')

In [431]:
agg_support.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155313 entries, 0 to 155312
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        155313 non-null  object 
 1   0102_2019  3501 non-null    float32
 2   0102_2020  4080 non-null    float32
 3   0102_2021  6547 non-null    float32
 4   0103_2019  10883 non-null   float32
 5   0103_2020  121553 non-null  float32
 6   0103_2021  82513 non-null   float32
 7   0105_2019  109 non-null     float32
 8   0105_2020  169 non-null     float32
 9   0105_2021  132 non-null     float32
dtypes: float32(9), object(1)
memory usage: 6.5+ MB


- есть странные min значения поддержки, средннее и медиана вроде адекватны

In [432]:
agg_support.describe()

,0102_2019,0102_2020,0102_2021,0103_2019,0103_2020,0103_2021,0105_2019,0105_2020,0105_2021
count,3.501000e+03,4.080000e+03,6.547000e+03,1.088300e+04,1.215530e+05,8.251300e+04,1.090000e+02,1.690000e+02,1.320000e+02
mean,2.635288e+07,2.326347e+07,2.094885e+07,4.842728e+06,7.846709e+05,8.063798e+05,2.085961e+07,1.177597e+07,1.250412e+07
std,7.375410e+07,5.957740e+07,6.949218e+07,1.512123e+07,7.304072e+06,5.117376e+06,4.013788e+07,2.027826e+07,1.904094e+07
min,4.575000e+03,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e-02,1.000000e-01,4.878713e+04,1.116072e+04,3.690000e+05
25%,3.000000e+06,3.000000e+06,2.750000e+06,2.794546e+05,4.852000e+04,2.558400e+04,2.352042e+06,2.394000e+06,4.181598e+06
50%,7.974450e+06,8.500000e+06,7.500000e+06,1.061325e+06,1.213000e+05,7.675200e+04,7.252944e+06,5.000000e+06,6.547269e+06
75%,2.100000e+07,2.100000e+07,1.950988e+07,3.591858e+06,3.396400e+05,2.430480e+05,1.627856e+07,1.017734e+07,1.254980e+07
max,1.912327e+09,1.844604e+09,2.674489e+09,4.968001e+08,1.794792e+09,4.561338e+08,2.292614e+08,1.267226e+08,1.623030e+08


In [433]:
agg_support.shape

(155313, 10)

In [434]:
agg_support.inn.nunique()

155313

- отдельно аггрегируем - были ли нарушения условий поддержки (просто сумма по годам)
- хотя можно было одной таблицей (values списком, aggfunc словарём), но если что - проще убрать

In [435]:
is_breach_agg = support_msp.groupby(['inn', 'support_year'])['is_breach'].sum().reset_index()

In [436]:
is_breach_agg.head(3)

,inn,support_year,is_breach
0,0101000021,2020,0
1,0101000021,2021,0
2,0101000078,2020,0


In [437]:
pivot_is_breach =  pd.pivot_table(is_breach_agg, values='is_breach', 
               index='inn', columns=['support_year'], aggfunc=sum, fill_value=0).reset_index()

In [438]:
pivot_is_breach.head()

support_year,inn,2019,2020,2021
0,0101000021,0,0,0
1,0101000078,0,0,0
2,0101000832,0,0,0
3,0101002861,0,0,0
4,0101003086,0,0,0


In [439]:
pivot_is_breach.loc[pivot_is_breach[2020] > 0].shape

(185, 4)

- переименовываем столбцы и приводим типы

In [440]:
pivot_is_breach.rename(columns={2019: 'breach_2019', 2020: 'breach_2020', 2021: 'breach_2021'}, 
                      inplace=True)

In [442]:
pivot_is_breach.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220624 entries, 0 to 220623
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   inn          220624 non-null  object
 1   breach_2019  220624 non-null  int8  
 2   breach_2020  220624 non-null  int8  
 3   breach_2021  220624 non-null  int8  
dtypes: int8(3), object(1)
memory usage: 2.3+ MB


#### - соединяем и заполняем пропуски нулями

In [443]:
data.shape

(1337248, 71)

In [444]:
%%time
data = data.merge(agg_support, how='left', on='inn')
data = data.merge(pivot_is_breach, how='left', on='inn')

Wall time: 4.04 s


In [445]:
data.shape

(1337248, 83)

In [446]:
agg_col

['0102_2019',
 '0102_2020',
 '0102_2021',
 '0103_2019',
 '0103_2020',
 '0103_2021',
 '0105_2019',
 '0105_2020',
 '0105_2021']

In [448]:
for col in agg_col + ['breach_2019', 'breach_2020', 'breach_2021']:
    data[col].fillna(0, inplace=True)

In [348]:
for col in ['breach_2019', 'breach_2020', 'breach_2021']:
    data[col] = pd.to_numeric(data[col], downcast='integer')

#### - следующая таблица - debt - налоговая задолженность

In [450]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)

In [454]:
connect.close()

In [451]:
columns = [
    'inn', 
#     'status_date',  # [datetime.date(2021, 12, 31)]
    'arrears', 
    'penalties', 
    'fines', 
]

In [452]:
debt = pd.read_sql(sql.SQL('SELECT {} FROM debt;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

In [453]:
debt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816569 entries, 0 to 816568
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        816569 non-null  object 
 1   arrears    816569 non-null  float64
 2   penalties  816569 non-null  float64
 3   fines      816569 non-null  float64
dtypes: float64(3), object(1)
memory usage: 24.9+ MB


- просто приведём типы, данные уже аггрегированы

In [455]:
for col in debt.columns[1:]:
    debt[col] = pd.to_numeric(debt[col], downcast='float')

In [456]:
debt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816569 entries, 0 to 816568
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   inn        816569 non-null  object 
 1   arrears    816569 non-null  float32
 2   penalties  816569 non-null  float32
 3   fines      816569 non-null  float32
dtypes: float32(3), object(1)
memory usage: 15.6+ MB


#### - соединяем и заполняем пропуски 0

In [457]:
data.shape

(1337248, 83)

In [458]:
%%time
data = data.merge(debt, how='left', on='inn')

Wall time: 2.93 s


In [459]:
data.shape

(1337248, 86)

In [460]:
for col in debt.columns[1:]:
    data[col].fillna(0, inplace=True)

#### - таблица tax_offense - налоговые нарушения

In [462]:
connect = psycopg2.connect(dbname="fns_data", user="user", password='new_password', host="localhost", port=5432)

In [465]:
connect.close()

In [463]:
columns = [
    'inn', 
#     'offense_date',  # одна для всех - [datetime.date(2021, 12, 31)]
    'fine_amount', 
]

In [464]:
tax_offense = pd.read_sql(sql.SQL('SELECT {} FROM tax_offense;')\
                            .format(sql.SQL(',').join(map(sql.Identifier, columns))), connect)

In [466]:
tax_offense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62723 entries, 0 to 62722
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   inn          62723 non-null  object 
 1   fine_amount  62723 non-null  float64
dtypes: float64(1), object(1)
memory usage: 980.2+ KB


In [467]:
tax_offense.fine_amount = pd.to_numeric(tax_offense.fine_amount, downcast='float')

#### - соединяем

In [469]:
%%time
data = data.merge(tax_offense, how='left', on='inn')

Wall time: 1.96 s


In [471]:
data.fine_amount.fillna(0, inplace=True)

#### -  inn_disqual - просто флаг
- Юридические лица, в состав исполнительных органов которых входят дисквалифицированные лица

        G3 здесь - инн

In [473]:
with_d_person = pd.read_csv('raw_data/with_disqual_person.csv', sep=';', dtype={'G3': str})
with_d_person.head(3)

,G1,G2,G3,G4,G5
0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КАРТЕН""",1175749002263,5751060080,575101001,"302030,ОРЛОВСКАЯ ОБЛАСТЬ,,ОРЁЛ ГОРОД,,МОСКОВСК..."
1,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ВЕПРЬ""",1087205000541,7222018450,722201001,"627500,ТЮМЕНСКАЯ ОБЛАСТЬ,СОРОКИНСКИЙ РАЙОН,,БО..."
2,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САМАН""",1190280053732,0272913945,27201001,"450095,БАШКОРТОСТАН РЕСПУБЛИКА,,УФА ГОРОД,,МАЙ..."


In [474]:
# все инн уникальные
with_d_person.G3.nunique()

4916

In [475]:
inn_disqual = tuple(with_d_person.G3)

In [476]:
data['disqual_person'] = 1

In [477]:
%%time
data.disqual_person.where(data.inn.isin(inn_disqual), 0, inplace=True)

Wall time: 144 ms


- бессмысленный признак, его слишком мало?
- но пусть пока побудет

In [478]:
data.disqual_person.value_counts()

0    1337088
1        160
Name: disqual_person, dtype: int64

In [ ]:
data.disqual_person = pd.to_numeric(data.disqual_person, downcast='int')

- временно сохраняю в файл

In [150]:
# with open('parsed_data/data.pickle', 'wb') as f:
#     pickle.dump(data, f)

In [237]:
with open('parsed_data/data.pickle', 'rb') as f:
    data = pickle.load(f)

#### - добавляем groc_and_fuels

In [13]:
# пропуски и уникальные коды регионов из data
data.region_code.isna().sum(), np.sort(data.region_code.unique())

(0,
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
        69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 86, 87, 89, 91, 92,
        95], dtype=int8))

- 93 соотв 92, 95 соотв 20, 94 - нет данных
- здесь только код 95, переименовывать в столбце не хочу - просто добавлю значение отдельно

In [15]:
data.loc[data.region_code == '94']

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,0105_2020,0105_2021,breach_2019,breach_2020,breach_2021,arrears,penalties,fines,fine_amount,disqual_person


In [117]:
prices_by_regions = pd.read_csv('raw_data/prices_by_regions/data-20220331-structure-20220331.csv', sep=';')

In [118]:
# отбираем и переименовываем столбцы
groc_and_fuels = prices_by_regions[['GA', 'G2', 'G35', 'G36', 'G37', 'G70', 'G71', 'G72', 'G73', 
                                    'G74', 'G75', 'G76', 'G77']]
groc_and_fuels.columns = ['region_code', 'observ_date', 'cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']

In [120]:
groc_and_fuels.head(3)

,region_code,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume
0,1,01.01.2019,"5023,51","4301,36","5001,97","42,96","46,87","50,2","45,53","103778,69","56968,84","121,19","13973,49"
1,1,02.01.2019,"5086,32",4354,"5064,53","42,98","46,92","50,3","45,38","185820,84","119512,26","95,09","39283,9"
2,1,03.01.2019,"4930,56","4214,3","4911,48","41,13","46,86","50,77","45,46","219913,38","126519,19","107,82","45498,67"


In [121]:
groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100810 entries, 0 to 100809
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   region_code              100810 non-null  int64 
 1   observ_date              100810 non-null  object
 2   cons_basket_working_age  100810 non-null  object
 3   cons_basket_pens         100810 non-null  object
 4   cons_basket_child        100810 non-null  object
 5   AI92_cost                100810 non-null  object
 6   AI95_cost                100810 non-null  object
 7   AI98_cost                100810 non-null  object
 8   diesel_cost              100810 non-null  object
 9   AI92_volume              100810 non-null  object
 10  AI95_volume              100810 non-null  object
 11  AI98_volume              100810 non-null  object
 12  diesel_volume            100810 non-null  object
dtypes: int64(1), object(12)
memory usage: 10.0+ MB


In [122]:
# заменяем запятую на точку и приводим к float
for col in groc_and_fuels.columns[2:]:
    groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)

# добавляем год (чтобы усреднить за период)
groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year
groc_and_fuels.head(3)

C:\Users\013\AppData\Local\Temp/ipykernel_11120/1479746939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels[col] = groc_and_fuels[col].apply(lambda x: x.replace(',', '.')).astype(float)
C:\Users\013\AppData\Local\Temp/ipykernel_11120/1479746939.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groc_and_fuels['observ_year'] = pd.to_datetime(groc_and_fuels['observ_date']).dt.year


,region_code,observ_date,cons_basket_working_age,cons_basket_pens,cons_basket_child,AI92_cost,AI95_cost,AI98_cost,diesel_cost,AI92_volume,AI95_volume,AI98_volume,diesel_volume,observ_year
0,1,01.01.2019,5023.51,4301.36,5001.97,42.96,46.87,50.20,45.53,103778.69,56968.84,121.19,13973.49,2019
1,1,02.01.2019,5086.32,4354.00,5064.53,42.98,46.92,50.30,45.38,185820.84,119512.26,95.09,39283.90,2019
2,1,03.01.2019,4930.56,4214.30,4911.48,41.13,46.86,50.77,45.46,219913.38,126519.19,107.82,45498.67,2019


In [123]:
groc_and_fuels.observ_year.unique()

array([2019, 2020, 2021, 2022], dtype=int64)

In [124]:
# оставляем только 21, 20 и 19 гг.
groc_and_fuels = groc_and_fuels.loc[groc_and_fuels.observ_year.isin((2021, 2020, 2019))]

In [125]:
# заменяем 0 на None - т.е. не цена 0, а отсутствие наблюдений
groc_and_fuels = groc_and_fuels.replace(0, None)

In [126]:
# средняя стоимость за период по регионам
avg_groc_and_fuels = groc_and_fuels.groupby(['region_code', 'observ_year'], as_index=False)[['cons_basket_working_age', 
                         'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 'AI95_cost', 
                         "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
                         "AI98_volume", 'diesel_volume']].mean()

- разворачиваем таблицу - чтобы получить столбцы по годам
- плюс 33 столбца

In [127]:
val_columns = ['cons_basket_working_age', 'cons_basket_pens', 'cons_basket_child', 'AI92_cost', 
               'AI95_cost', "AI98_cost", 'diesel_cost', 'AI92_volume', 'AI95_volume', 
               "AI98_volume", 'diesel_volume']

In [128]:
pivot_groc_and_fuels =  pd.pivot_table(avg_groc_and_fuels, values=val_columns, 
               index='region_code', columns=['observ_year'], aggfunc='mean').reset_index()

In [129]:
pivot_groc_and_fuels.head()

region_code  AI92_cost                         AI92_volume  \
observ_year                   2019       2020       2021          2019   
0                     1  43.332219  43.547923  44.553863  2.211384e+05   
1                     2  41.124986  42.006940  43.900411  2.018062e+06   
2                     3  42.091425  43.820874  45.771123  4.270480e+05   
3                     4  42.244000  41.944891  45.373288  8.576184e+04   
4                     5  42.700493  42.862760  45.421041  1.113027e+04   

                                         AI95_cost                        ...  \
observ_year          2020          2021       2019       2020       2021  ...   
0            1.874788e+05  2.081576e+05  48.051562  48.533852  50.663260  ...   
1            1.795260e+06  1.718155e+06  44.246466  45.193770  47.296274  ...   
2            2.976584e+05  3.115107e+05  43.764466  45.792158  48.114411  ...   
3            6.974820e+04  9.066889e+04  44.202932  44.186038  47.365479  ...   
4            7.508397e+03  2.065066e+04  45.790740  46.983880  50.277397  ...   

            cons_basket_pens cons_basket_working_age               \
observ_year             2021                    2019         2020   
0                5031.818411             5005.002466  5146.416749   
1                4804.212411             4637.134110  4863.667842   
2                5167.355589             5084.651507  5433.417869   
3                4772.020712             4623.268630  4949.242541   
4                5594.561616             5007.779616  5325.073033   

                         diesel_cost                        diesel_volume  \
observ_year         2021        2019       2020       2021           2019   
0            5914.448685   45.404822  46.012650  47.691973   65073.028384   
1            5657.522795   45.621288  46.787104  48.158740  322267.884164   
2            6080.860192   47.587370  50.227650  53.012384  103348.248466   
3            5604.221205   46.775726  48.395792  50.237151   16605.699342   
4            6542.970658   42.806658  44.108743  46.466849    1374.417068   

                                           
observ_year           2020           2021  
0             60822.033087   69281.618521  
1            286174.082486  277899.671616  
2             79717.239016  103949.519370  
3             13378.189536   21544.138055  
4              2379.751612    5120.080192  

[5 rows x 34 columns]

- избавляемся от мультииндекса в названиях столбцов

In [130]:
pivot_groc_and_fuels.columns = ['region_code'] + [f'{el[0]}_{str(el[1])}' for el in pivot_groc_and_fuels.columns.values[1:]]

In [131]:
pivot_groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   region_code                   85 non-null     int64  
 1   AI92_cost_2019                85 non-null     float64
 2   AI92_cost_2020                85 non-null     float64
 3   AI92_cost_2021                85 non-null     float64
 4   AI92_volume_2019              85 non-null     float64
 5   AI92_volume_2020              85 non-null     float64
 6   AI92_volume_2021              85 non-null     float64
 7   AI95_cost_2019                85 non-null     float64
 8   AI95_cost_2020                85 non-null     float64
 9   AI95_cost_2021                85 non-null     float64
 10  AI95_volume_2019              85 non-null     float64
 11  AI95_volume_2020              85 non-null     float64
 12  AI95_volume_2021              85 non-null     float64
 13  AI98_co

- добавляем строку для кода региона 95 == 20

In [134]:
val = pivot_groc_and_fuels.loc[pivot_groc_and_fuels.region_code == 20, pivot_groc_and_fuels.columns[1:]].values[0]

In [136]:
pivot_groc_and_fuels.loc[len(pivot_groc_and_fuels.index)] = [95, *val]

In [137]:
pivot_groc_and_fuels

,region_code,AI92_cost_2019,AI92_cost_2020,AI92_cost_2021,AI92_volume_2019,AI92_volume_2020,AI92_volume_2021,AI95_cost_2019,AI95_cost_2020,AI95_cost_2021,...,cons_basket_pens_2021,cons_basket_working_age_2019,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021
0,1.0,43.332218,43.547924,44.553864,2.211384e+05,1.874788e+05,2.081576e+05,48.051563,48.533852,50.663261,...,5031.818359,5005.002441,5146.416992,5914.448730,45.404823,46.012650,47.691971,65073.027344,60822.031250,69281.617188
1,2.0,41.124985,42.006939,43.900410,2.018062e+06,1.795260e+06,1.718155e+06,44.246468,45.193771,47.296272,...,4804.212402,4637.134277,4863.667969,5657.522949,45.621288,46.787106,48.158741,322267.875000,286174.093750,277899.656250
2,3.0,42.091423,43.820873,45.771122,4.270480e+05,2.976584e+05,3.115108e+05,43.764465,45.792160,48.114410,...,5167.355469,5084.651367,5433.417969,6080.860352,47.587368,50.227650,53.012383,103348.250000,79717.242188,103949.515625
3,4.0,42.243999,41.944889,45.373287,8.576184e+04,6.974820e+04,9.066888e+04,44.202930,44.186039,47.365479,...,4772.020508,4623.268555,4949.242676,5604.221191,46.775726,48.395794,50.237152,16605.699219,13378.189453,21544.138672
4,5.0,42.700493,42.862759,45.421040,1.113027e+04,7.508397e+03,2.065066e+04,45.790741,46.983879,50.277397,...,5594.561523,5007.779785,5325.073242,6542.970703,42.806656,44.108742,46.466850,1374.417114,2379.751709,5120.080078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,87.0,55.823067,57.110657,60.320110,4.700495e+04,5.039137e+03,4.162756e+03,58.500656,59.743168,63.000000,...,10695.199219,10346.186523,11671.370117,12622.214844,58.889999,58.889999,61.667698,62312.179688,62312.179688,7231.630859
82,89.0,41.047043,42.074646,45.385700,1.690373e+05,1.127314e+05,1.606221e+05,43.152821,44.184971,47.579098,...,6041.364746,5980.732910,6262.968262,7081.270996,49.568382,50.502186,52.313972,30115.738281,20314.312500,30925.708984
83,91.0,45.643341,45.005901,48.836437,4.004605e+05,3.554006e+05,4.437638e+05,49.274796,50.027569,52.795563,...,5208.303711,5177.835938,5387.308105,6091.378906,50.184933,49.070381,52.189369,285398.875000,238884.031250,284649.468750
84,92.0,45.714985,45.112293,48.618137,8.431954e+04,8.157254e+04,9.469902e+04,49.335533,50.032131,52.723454,...,5498.511719,5421.074219,5430.147949,6434.974609,50.312576,49.171940,52.279808,73136.218750,73454.476562,73371.609375


- приводим типы

In [141]:
for col in pivot_groc_and_fuels.columns[1:]:
    pivot_groc_and_fuels[col] = pd.to_numeric(pivot_groc_and_fuels[col], downcast='float')
    
pivot_groc_and_fuels.region_code = pd.to_numeric(pivot_groc_and_fuels.region_code, downcast='integer')

In [142]:
pivot_groc_and_fuels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 0 to 85
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   region_code                   86 non-null     int8   
 1   AI92_cost_2019                86 non-null     float32
 2   AI92_cost_2020                86 non-null     float32
 3   AI92_cost_2021                86 non-null     float32
 4   AI92_volume_2019              86 non-null     float32
 5   AI92_volume_2020              86 non-null     float32
 6   AI92_volume_2021              86 non-null     float32
 7   AI95_cost_2019                86 non-null     float32
 8   AI95_cost_2020                86 non-null     float32
 9   AI95_cost_2021                86 non-null     float32
 10  AI95_volume_2019              86 non-null     float32
 11  AI95_volume_2020              86 non-null     float32
 12  AI95_volume_2021              86 non-null     float32
 13  AI98_co

#### - соединяем

In [147]:
%%time
data = data.merge(pivot_groc_and_fuels, how='left', on='region_code')

Wall time: 1.55 s


In [148]:
data.shape

(1337248, 121)

#### численность по регионам - здесь придётся немного сложнее

##### - для начала максимально заполню пропуски по городам - если нет поселения а есть регион = город

In [239]:
# для тестов
addresses_col = ['region', 'city', 'settlement']
addresses  = data[addresses_col]

- приводим к одному регистру

In [240]:
for col in addresses_col:
    addresses[col] = addresses[col].apply(lambda x: x.title() if x else x)

C:\Users\013\AppData\Local\Temp/ipykernel_11120/3677381831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses[col] = addresses[col].apply(lambda x: x.title() if x else x)


In [241]:
addresses

,region,city,settlement
0,Волгоградская,Волгоград,None
1,Волгоградская,Волгоград,None
2,Волгоградская,Волгоград,None
3,Волгоградская,Волгоград,None
4,Волгоградская,Волгоград,None
...,...,...,...
1337243,Ямало-Ненецкий,Салехард,None
1337244,Ямало-Ненецкий,Лабытнанги,None
1337245,Свердловская,Екатеринбург,None
1337246,Ямало-Ненецкий,Салехард,None


-  нет ни города ни поселения

In [242]:
addresses.loc[addresses.city.isna() & addresses.settlement.isna()].shape, addresses.loc[addresses.city.isna() & addresses.settlement.isna()].region.unique()

((334976, 3),
 array(['Московская', 'Москва', 'Санкт-Петербург', 'Ленинградская',
        'Хабаровский', 'Севастополь', 'Самарская', 'Оренбургская',
        'Костромская', 'Иркутская', 'Татарстан', 'Ульяновская',
        'Пензенская', 'Новгородская', 'Ростовская', 'Пермский',
        'Удмуртская', 'Свердловская', 'Липецкая', 'Ивановская',
        'Башкортостан', 'Новосибирская', 'Приморский', 'Саратовская',
        'Тюменская', 'Тамбовская', 'Тверская', 'Карелия', 'Забайкальский',
        'Мурманская', 'Воронежская', 'Краснодарский', 'Красноярский',
        'Белгородская', 'Ставропольский', 'Нижегородская', 'Курганская',
        'Томская', 'Тульская', 'Марий Эл', 'Астраханская', 'Калужская',
        'Смоленская', 'Владимирская', 'Камчатский', 'Коми', 'Челябинская',
        'Калининградская', 'Бурятия', 'Алтай', 'Крым', 'Кировская',
        'Сахалинская', 'Рязанская', 'Курская', 'Псковская',
        'Архангельская', 'Ханты-Мансийский Автономный Округ - Югра',
        'Чувашская Республи

- нет города, есть поселения

In [243]:
addresses.loc[addresses.city.isna() & ~addresses.settlement.isna()].region.unique()

array(['Московская', 'Москва', 'Алтайский', 'Новгородская', 'Алтай',
       'Саха /Якутия/', 'Нижегородская', 'Ненецкий', 'Архангельская',
       'Адыгея', 'Брянская', 'Приморский', 'Сахалинская', 'Кировская',
       'Ставропольский', 'Санкт-Петербург', 'Псковская', 'Ленинградская',
       'Бурятия', 'Забайкальский', 'Волгоградская', 'Краснодарский',
       'Кемеровская Область - Кузбасс', 'Новосибирская', 'Хабаровский',
       'Томская', 'Самарская', 'Смоленская', 'Свердловская', 'Чеченская',
       'Ханты-Мансийский Автономный Округ - Югра', 'Воронежская',
       'Красноярский', 'Рязанская', 'Мордовия', 'Тамбовская',
       'Оренбургская', 'Пермский', 'Чукотский', 'Ярославская',
       'Костромская', 'Татарстан', 'Башкортостан',
       'Чувашская Республика -', 'Хакасия', 'Крым', 'Тульская',
       'Саратовская', 'Северная Осетия - Алания', 'Пензенская',
       'Владимирская', 'Севастополь', 'Ульяновская', 'Калужская',
       'Омская', 'Вологодская', 'Челябинская', 'Ростовская',
    

- ни региона и города ни поселения

In [244]:
# ну, хоть что-то, да есть для каждой строки
addresses.loc[addresses.city.isna() & addresses.settlement.isna() & addresses.region.isna()]

,region,city,settlement


- для тех, у кого нет города, но в графе регион - название города (это для крупных городов, где город имеет свой код региона) - заполняем поле city

        'Москва'
        'Санкт-Петербург'
        'Севастополь'

In [245]:
# это интересно - у них всех код региона 50 - московская область
# те с адресом всё нетривиально
data.loc[data.region == 'БАЙКОНУР']

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,cons_basket_pens_2021,cons_basket_working_age_2019,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021
639112,9901033939,"ООО ""СКАЗКА""",468320,БАЙКОНУР,None,None,47.25,12300,ACTIVE,2008-12-16,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639113,9901034629,"ООО ""ПРОФИ-СОФТ""",468321,БАЙКОНУР,None,None,62.0,12300,ACTIVE,2012-01-23,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639114,9901035171,"ООО ""ВЕНТА""",468320,БАЙКОНУР,None,None,41.20,12300,ACTIVE,2014-01-30,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639115,9901035647,"ООО ""РС №2""",468320,БАЙКОНУР,None,None,43.11,12300,ACTIVE,2015-03-12,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
639116,9901034636,"ООО ""ОРЕХ""",468320,БАЙКОНУР,None,None,38.32.2,12300,ACTIVE,2012-02-13,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762447,9901001535,"ООО ""ФЕРМЕРСКОЕ ХОЗЯЙСТВО БОЛАШАК""",468325,БАЙКОНУР,None,None,68.20,12300,ACTIVE,1996-03-19,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
762448,9901001888,ПК ГСК № 24,468320,БАЙКОНУР,None,None,94.99,20101,ACTIVE,1996-04-22,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
762449,9901000940,"ООО ""ТЕХНОСЕРВИС""",468325,БАЙКОНУР,None,None,47.30,12300,ACTIVE,1996-11-29,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25
762450,9901001165,"ООО ""БУТЯ-БАЙКОНУР""",468320,БАЙКОНУР,None,None,47.75,12300,ACTIVE,1996-03-01,...,5782.85498,5762.977051,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25


In [246]:
data.loc[data.region == 'БАЙКОНУР'].region_code.unique()

array([50], dtype=int8)

- т.е. можно заполнить почти все пропуски города (328 300 из 334 976)

In [247]:
# условие для заполнения
cond = addresses.city.isna() & addresses.region.isin(('Москва', 'Санкт-Петербург', 'Севастополь'))

In [248]:
addresses.city = addresses.city.where(~cond, addresses.region)

C:\Users\013\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [249]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337248 entries, 0 to 1337247
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   region      1337209 non-null  object
 1   city        1217481 non-null  object
 2   settlement  139215 non-null   object
dtypes: object(3)
memory usage: 40.8+ MB


In [250]:
# осталось 13 тыс с только областью - неплохо
addresses.loc[addresses.city.isna() & addresses.settlement.isna()]

,region,city,settlement
7,Московская,None,None
10,Московская,None,None
14,Московская,None,None
16,Московская,None,None
17,Московская,None,None
...,...,...,...
1335934,Московская,None,None
1336161,Московская,None,None
1336185,Московская,None,None
1336238,Московская,None,None


##### - переходим к основным данным
    - приводим к одному виду
    - заполняем город, для кого можно

In [251]:
for col in addresses_col:
    data[col] = data[col].apply(lambda x: x.title() if x else x)

In [252]:
data[addresses_col].head()

,region,city,settlement
0,Волгоградская,Волгоград,None
1,Волгоградская,Волгоград,None
2,Волгоградская,Волгоград,None
3,Волгоградская,Волгоград,None
4,Волгоградская,Волгоград,None


In [253]:
cond = data.city.isna() & data.region.isin(('Москва', 'Санкт-Петербург', 'Севастополь'))

In [254]:
data.city = data.city.where(~cond, data.region)

In [255]:
data[addresses_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337248 entries, 0 to 1337247
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   region      1337209 non-null  object
 1   city        1217481 non-null  object
 2   settlement  139215 non-null   object
dtypes: object(3)
memory usage: 40.8+ MB


In [256]:
data.loc[data.region == 'Москва', addresses_col].head(3)

,region,city,settlement
15,Москва,Москва,None
19,Москва,Москва,None
24,Москва,Москва,None


- заменяем некоторые названия городов (см предыдущие ноутбуки)

In [257]:
city_replace = {
    'Королев': 'Королёв',
    'Белохолуницкое Городское Поселение': 'Белохолуницкое',
    'Мо Сельское Поселение Следневское': 'Следневское',
    'Камень-На-Оби': 'Камень-на-Оби',
    'Широкобуеракское Мо': 'Широкобуеракское',
    'Новопушкинское Мо': 'Новопушкинское',
    'Ростов-На-Дону': 'Ростов-на-Дону',
    'Сольцы 2': 'Сольцы',
    'Буденновск': 'Будённовск',
    'Городское Поселение Снегири': 'Снегири',
    'Калач-На-Дону': 'Калач-на-Дону',
    'Славянск-На-Кубани': 'Славянск-на-Кубани',
    'Терсинское Мо': 'Терсинское',
    'Николаевск-На-Амуре': 'Николаевск-на-Амуре',
    'Комсомольск-На-Амуре': 'Комсомольск-на-Амуре',
}

In [258]:
data.city.replace(city_replace, inplace=True)

- названия settlements так вычистить не получится - их слишком много и слишком мало совпадений

##### - и только теперь переходим к данным переписи (данные за 20 год)

In [317]:
vpn_data = pd.read_excel('raw_data/tab-5_VPN-2020.xlsx', header=3)
population = vpn_data[['Unnamed: 0', 'Unnamed: 1']].rename(columns={'Unnamed: 0': 'locality', 'Unnamed: 1': 'population'})

In [318]:
population.head()

,locality,population
0,Российская Федерация,147182123
1,Городское население,110075322
2,Сельское население,37106801
3,Центральный федеральный округ,40334532
4,Городское население,33118003


- уберём пропуски

In [319]:
population.loc[population.population == '-']

,locality,population
237,Сельское население,-
443,Сельское население,-
648,Сельское население,-
922,прочие сельские населенные пункты,-
971,прочие сельские населенные пункты,-
...,...,...
27398,Сельское население,-
27510,Сельское население,-
27587,пгт Беличан,-
27605,пгт Верхний Ат-Урях,-


In [320]:
population = population.loc[population.population != '-']

- создадим словарь город-численность

In [262]:
# порядок важен - в городах сперва ищем город
# выражение не оптимально
def get_population(name):
    val = population.loc[population['locality']\
          .str \
          .contains(rf'г?. {name}\b|пгт?. {name}\b|[Сс]ело {name}\b|пос[ёе]лок {name}\b|п. {name}\b|[Сс]ельское поселение {name}\b|^{name}\b сельское поселение|Поселение {name}\b|[Дд]еревня {name}\b|город {name}\b')]\
          .population.values
    return val[0] if val.any() else None

In [265]:
cities = data.city.unique()

In [269]:
cities, len(cities)

(array(['Волгоград', None, 'Одинцово', ..., 'Новое Токсово', 'Вишневский',
        'Юхнов-1'], dtype=object),
 1261)

In [267]:
%%time
city_to_population = {key: get_population(key) for key in cities}

Wall time: 2min 33s


In [270]:
city_to_population

{'Волгоград': 1028036,
 None: None,
 'Одинцово': 180530,
 'Москва': 13010112,
 'Каменск-Шахтинский': 86365,
 'Калуга': 358960,
 'Краснознаменск': 43868,
 'Звенигород': 35842,
 'Бронницы': 21831,
 'Красногорск': 316194,
 'Видное': 101490,
 'Долгопрудный': 120907,
 'Руза': 16014,
 'Александров': 57053,
 'Зеленоград': None,
 'Кубинка': 23146,
 'Голицыно': 22733,
 'Балашиха': 543432,
 'Химки': 257128,
 'Апрелевка': 35514,
 'Волоколамск': 26389,
 'Талдом': 17317,
 'Люберцы': 348535,
 'Барнаул': 698603,
 'Новосибирск': 1633595,
 'Элиста': 107079,
 'Бийск': 191963,
 'Кемерово': 557119,
 'Златоуст': 161774,
 'Новоалтайск': 73049,
 'Алейск': 25380,
 'Рубцовск': 126834,
 'Горно-Алтайск': 65342,
 'Ростов-на-Дону': 1142162,
 'Воронеж': 1057681,
 'Раменское': 1177,
 'Ставрополь': 547601,
 'Самара': 1173393,
 'Калининград': 490449,
 'Надым': 45973,
 'Подольск': 353094,
 'Казань': 1308660,
 'Чехов': 148804,
 'Мытищи': 287343,
 'Сортавала': 14867,
 'Королёв': 228095,
 'Ейск': 82943,
 'Белгород': 33997

- то же самое для settlememt, но гораздо дольше (так как уникальных около 20 тыс) и гораздо менее эффективно - будет всего около 4 тыс значений
- остальные придётся заполнять (например, медианой всех значений менее 3 тыс) - именно для таких населённых пунктов нет данных

In [272]:
def get_population_settl(name):
    val = population.loc[population['locality']\
          .str \
          .contains(rf'пгт?. {name}\b|[Сс]ело {name}\b|пос[ёе]лок {name}\b|п?. {name}\b|[Сс]ельское поселение {name}\b|^{name}\b сельское поселение|Поселение {name}\b|[Дд]еревня {name}\b')]\
          .population.values
    return val[0] if val.any() else None

In [271]:
settlements = data.settlement.unique()

Wall time: 35min 58s

In [ ]:
# %%time
# settlement_to_population = {key: get_population_settl(key) for key in settlements}

In [331]:
# with open('parsed_data/settlement_to_population.pickle', 'wb') as f:
#     pickle.dump(settlement_to_population, f)

In [324]:
# просто прочитаю из файла
with open('parsed_data/settlement_to_population.pickle', 'rb') as f:
    settlement_to_population = pickle.load(f)

- словарь с численностью для регионов - самый полный и чистый

In [283]:
# словарь перевода кода региона в имя региона
# вообще мб путаницы: разные версии кодов - для каких-то регионов несколько кодов, некоторые 
# коды объединяются, используются временно... переименовываются...
region_code_to_name = {
#    0: 'Unknown',  # неизвестный регион
    1: 'Республика Адыгея',
    2: 'Республика Башкортостан',
    3: 'Республика Бурятия',
    4: 'Республика Алтай',
    5: 'Республика Дагестан',
    6: 'Республика Ингушетия',
    7: 'Кабардино-Балкарская Республика',
    8: 'Республика Калмыкия',
    9: 'Карачаево-Черкесская Республика',
    10: 'Республика Карелия',
    11: 'Республика Коми',
    12: 'Республика Марий Эл',
    13: 'Республика Мордовия',
    14: 'Республика Саха (Якутия)',
    15: 'Республика Северная Осетия-Алания',
    16: 'Республика Татарстан',
    17: 'Республика Тыва',
    18: 'Удмуртская Республика',
    19: 'Республика Хакасия',
    20: 'Чеченская Республика',
    21: 'Чувашская Республика',
    22: 'Алтайский край',
    23: 'Краснодарский край',
    24: 'Красноярский край',
    25: 'Приморский край',
    26: 'Ставропольский край',
    27: 'Хабаровский край',
    28: 'Амурская область',
    29: 'Архангельская область',
    30: 'Астраханская область',
    31: 'Белгородская область',
    32: 'Брянская область',
    33: 'Владимирская область',
    34: 'Волгоградская область',
    35: 'Вологодская область',
    36: 'Воронежская область',
    37: 'Ивановская область',
    38: 'Иркутская область',
    39: 'Калининградская область',
    40: 'Калужская область',
    41: 'Камчатский край',  # Камчатская область
    42: 'Кемеровская область – Кузбасс',
    43: 'Кировская область',
    44: 'Костромская область',
    45: 'Курганская область',
    46: 'Курская область',
    47: 'Ленинградская область',
    48: 'Липецкая область',
    49: 'Магаданская область',
    50: 'Московская область',
    51: 'Мурманская область',
    52: 'Нижегородская область',
    53: 'Новгородская область',
    54: 'Новосибирская область',
    55: 'Омская область',
    56: 'Оренбургская область',
    57: 'Орловская область',
    58: 'Пензенская область',
    59: 'Пермский край',  # Пермская область
    60: 'Псковская область',
    61: 'Ростовская область',
    62: 'Рязанская область',
    63: 'Самарская область',
    64: 'Саратовская область',
    65: 'Сахалинская область',
    66: 'Свердловская область',
    67: 'Смоленская область',
    68: 'Тамбовская область',
    69: 'Тверская область',
    70: 'Томская область',
    71: 'Тульская область',
    72: 'Тюменская область',
    73: 'Ульяновская область',
    74: 'Челябинская область',
    75: 'Забайкальский край',  # Читинская область
    76: 'Ярославская область',
    77: 'г. Москва - городское население',  # так в таблице, ищу точное совпадение
    78: 'г. Санкт-Петербург - городское население',
    79: 'Еврейская автономная область',
    83: 'Ненецкий автономный округ',
    86: 'Ханты-Мансийский автономный округ - Югра',
    87: 'Чукотский автономный округ',
    89: 'Ямало-Ненецкий автономный округ',
    90: 'Московская область',  # то же, что и код 50
    91: 'Республика Крым',
    92: 'г. Севастополь',
    93: 'Краснодарский край',  # == 23 Краснодарский край
#    94: #  == 0 - Байконур, данных нет
    95: 'Чеченская Республика'#  == 20 Чеченская респ.
}

In [278]:
np.sort(data.region_code.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 86, 87, 89, 91, 92,
       95], dtype=int8)

In [284]:
# словарик код - численность
get_p = lambda x: population.loc[population.locality == region_code_to_name[x], 'population'].values[0]
region_code_to_population = {key: get_p(key) for key in region_code_to_name}

In [286]:
region_code_to_population[1]

496934

#### - объединяем

In [288]:
data['region_population'] = data.region_code.replace(region_code_to_population)

In [291]:
data.loc[data.region_code == 1].head(3)

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,cons_basket_working_age_2019,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021,region_population
2239,0105018478,ООО ФИРМА ОШТЕН-1,385000,Адыгея,Майкоп,None,43.91,12300,ACTIVE,2001-06-07,...,5005.002441,5146.416992,5914.44873,45.404823,46.01265,47.691971,65073.027344,60822.03125,69281.617188,496934
2240,0105024440,"ООО ""КПЦ""",385019,Адыгея,Майкоп,None,96.09,12300,ACTIVE,1996-08-20,...,5005.002441,5146.416992,5914.44873,45.404823,46.01265,47.691971,65073.027344,60822.03125,69281.617188,496934
2241,0105027730,"ООО ФИРМА ""ЦЕНТР""",385009,Адыгея,Майкоп,None,77.39,12300,ACTIVE,1998-03-26,...,5005.002441,5146.416992,5914.44873,45.404823,46.01265,47.691971,65073.027344,60822.03125,69281.617188,496934


In [338]:
data['city_population'] = data.city.replace(city_to_population)

In [294]:
data.loc[data.city == 'Королёв'].head(3)

,inn,shortName,index,region,city,settlement,okved2,okopf,statusCode,registrationDate,...,cons_basket_working_age_2020,cons_basket_working_age_2021,diesel_cost_2019,diesel_cost_2020,diesel_cost_2021,diesel_volume_2019,diesel_volume_2020,diesel_volume_2021,region_population,city_population
1637,7703819290,"ООО ""КОМПАНИЯ ""МОСКОВСКИЕ КАНИКУЛЫ""",141075,Московская,Королёв,None,79.11,12300,ACTIVE,2014-10-09,...,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25,8524665,228095.0
9180,7701679369,"ООО ""КОНЬЯЧНЫЙ ДОМ ""АРМКО""",141080,Московская,Королёв,None,46.34.2,12300,ACTIVE,2006-09-13,...,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25,8524665,228095.0
11611,5638077275,"ПК ""АЛЬТЕРНА""",141065,Московская,Королёв,None,64.99,20100,ACTIVE,2020-08-20,...,6004.04834,6792.406738,45.575645,46.64019,48.816631,1720411.75,1383346.5,1870944.25,8524665,228095.0


In [339]:
%%time
data['settlement_population'] = data.settlement.replace(settlement_to_population)

Wall time: 4min 20s


- приводим типы

In [340]:
data.region_population = pd.to_numeric(data.region_population, downcast='integer')

In [341]:
for col in ['city_population', 'settlement_population']:
    data[col] = pd.to_numeric(data[col], downcast='float')

- итоговый датасет

In [349]:
data.info(verbose=True, show_counts=True, memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337248 entries, 0 to 1337247
Data columns (total 124 columns):
 #    Column                        Non-Null Count    Dtype         
---   ------                        --------------    -----         
 0    inn                           1337248 non-null  object        
 1    shortName                     1337248 non-null  object        
 2    index                         1333103 non-null  object        
 3    region                        1337209 non-null  object        
 4    city                          1217481 non-null  object        
 5    settlement                    139215 non-null   object        
 6    okved2                        1336263 non-null  object        
 7    okopf                         1337248 non-null  object        
 8    statusCode                    1337248 non-null  object        
 9    registrationDate              1337247 non-null  datetime64[ns]
 10   authorizedCapital             1281772 non-null  floa

In [350]:
# with open('parsed_data/data.pickle', 'wb') as f:
#     pickle.dump(data, f)

- больше 100 столбцов
- не все из них одинаково полезны
- достаточно много уйдёт после выбора цели и периода + после объединения/удаления/изменения сильно cкоррелированных признаков...